# Método CRISP-DM aplicado a la predicción de precios de hogares en el área metropolitana

## Entendimiento del Negocio


https://eafit-my.sharepoint.com/:w:/g/personal/lagonzala1_eafit_edu_co/ETHPB5HNXmZKqmOdtcFGZR4BHfAYmSyKaGVeLXXzUd0zPQ?e=2G8KqK


## Entendimiento de los datos

### Carga de datos:

In [125]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
from datetime import datetime
import warnings
plt.rcParams.update({'font.size': 14})
warnings.filterwarnings('ignore')


In [126]:
df = pd.read_json("readydata.json")
df.head()

,ID de la propiedad,Tipo de Operacion,Tipo de Inmueble,Departamento,Ciudad,Localidad Principal,Tipo de Localidad Principal,Metros Cuadrados,Metros Cuadrados Construidos,Estrato,...,Es Oficina?,Es Penthouse?,Acepta Permuta?,(paid quota)?,Fecha de Creacion de Oferta,Fecha de Actualización de Oferta,Dirección,URL,Precio con Administración Incluida,Precio
0,192788587,Venta,Apartamento,Antioquia,Bello,Niquia,neighbourhood,59.0,NaN,3,...,False,False,False,True,2025-08-25,2025-09-16,"Apartamento en Venta en Niquia, Bello",/apartamento-en-venta-en-niquia-bello/192788587,275000000,275000000
1,192805940,Venta,Apartamento,Antioquia,Bello,Andalucia,neighbourhood,64.9,64.9,3,...,False,False,False,True,2025-08-29,2025-08-29,"Apartamento en Venta en Andalucia, Bello",/apartamento-en-venta-en-andalucia-bello/19280...,280000000,280000000
2,192833958,Venta,Apartamento,Antioquia,Bello,Bello,city,67.0,NaN,3,...,False,False,False,True,2025-09-04,2025-10-30,Apartamento en Venta en Bello,/apartamento-en-venta-en-bello/192833958,260241000,260000000
3,192839470,Venta,Apartamento,Antioquia,Bello,Bello,city,72.0,72.0,3,...,False,False,False,True,2025-09-05,2025-09-20,Apartamento en Venta en Bello,/apartamento-en-venta-en-bello/192839470,433369521,433000000
4,192852231,Venta,Casa,Antioquia,Bello,Prado,neighbourhood,91.0,91.0,3,...,False,False,False,True,2025-09-09,2025-09-29,"Casa en Venta en Prado, Bello",/casa-en-venta-en-prado-bello/192852231,295000000,295000000


### Orígen de los datos
Antes de proceder, es importante comprender de dónde provienen estos datos.

Estos fueron extraídos de un portal web utilizando ténicas de web scraping, estos datos son disponibles públicamente. 

El web scraping no es infalible, se asume la existencia se ciertos elementos en las páginas que se visitan y, si el nombre de variables cambia, o si estas no aparecen, errores inesperados pueden suceder.

Es por esto que es necesario realizar un análsis exhaustivo en estas primeras fases, garantizando la integridad del proceso.

In [127]:
df.shape

(5827, 31)

In [128]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5827 entries, 0 to 5826
Data columns (total 31 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   ID de la propiedad                  5827 non-null   int64  
 1   Tipo de Operacion                   5827 non-null   object 
 2   Tipo de Inmueble                    5827 non-null   object 
 3   Departamento                        5827 non-null   object 
 4   Ciudad                              5827 non-null   object 
 5   Localidad Principal                 5823 non-null   object 
 6   Tipo de Localidad Principal         5823 non-null   object 
 7   Metros Cuadrados                    5785 non-null   float64
 8   Metros Cuadrados Construidos        4828 non-null   float64
 9   Estrato                             5827 non-null   int64  
 10  Latitud                             5827 non-null   float64
 11  Longitud                            5827 non-nul

### Análisis de calidad de los datos

#### Completitud

In [129]:
df.isna().sum()

ID de la propiedad                       0
Tipo de Operacion                        0
Tipo de Inmueble                         0
Departamento                             0
Ciudad                                   0
Localidad Principal                      4
Tipo de Localidad Principal              4
Metros Cuadrados                        42
Metros Cuadrados Construidos           999
Estrato                                  0
Latitud                                  0
Longitud                                 0
Antigüedad (Grupo)                     124
Cuartos (rooms)                          0
Dormitorios (bedrooms)                   0
Baños                                    0
Parqueaderos                             0
Piso                                   215
Tiene Garage?                            0
Es Proyecto?                          5814
Es Unidad Projecto?                      0
Es Oficina?                              0
Es Penthouse?                            0
Acepta Perm

Por lo que podemos ver, hay algunos registros vacíos, por lo tanto no está completo del todo. Sin embargo el dataset no tiene vacíos aparentes que puedan aportar más información acerca del precio del arriendo de los hogares.

#### Conformidad/Validez

In [130]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5827 entries, 0 to 5826
Data columns (total 31 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   ID de la propiedad                  5827 non-null   int64  
 1   Tipo de Operacion                   5827 non-null   object 
 2   Tipo de Inmueble                    5827 non-null   object 
 3   Departamento                        5827 non-null   object 
 4   Ciudad                              5827 non-null   object 
 5   Localidad Principal                 5823 non-null   object 
 6   Tipo de Localidad Principal         5823 non-null   object 
 7   Metros Cuadrados                    5785 non-null   float64
 8   Metros Cuadrados Construidos        4828 non-null   float64
 9   Estrato                             5827 non-null   int64  
 10  Latitud                             5827 non-null   float64
 11  Longitud                            5827 non-nul

**Análisis:** Que algunas columnas sean de tipo "object" es engañoso, object puede contener realmente cualquier cosa. Esto puede sugerir que en algunas columnas hay números donde no debe haber, o strings, o cualquier cosa.

Para ello se verificará lo siguiente:

1- Tipo de operacion: Todas las operaciones deben ser de tipo "Venta" si no hubo fallas en la extracción.

2- Tipo de Inmueble: Analizar los valores únicos de la columna y verificar sean como se espera.

3- Departamento: Todas las filas deberían tener como departamento "Antioquia". 

4- Ciudad: Se deben analizar los elementos únicos de la columna ciudad y contrastar con las ciudades del Área Metropolitana del Valle de Aburrá que se planea usar en el proyecto (Medellín, Envigado, Sabaneta, Itagüi, La Estrella y Bello).

5- Localidad Principal: Al momento de la extracción no es claro aún qué representaba esta etiqueta en el sistema del portal, por esto se hará un análisis de los valores únicos.

6- Tipo de Localidad Principal: Semejante a lo anterior.

7- Analizar la variable "Es Proyecto?" aparentemente booleana y verificar por qué se ha leído como float64.

8- Fechas: Nuevamente, object puede ser cucalquier cosa, se debe intentar la conversión a fecha y analizar si hay algun error por formato incorrecto.

9- Finalmente, analizar las variables booleanas y decidir si  son suficientemente relevantes.

** **Verificar tipo de Operación** **

In [131]:
# Tipo de operacion.

df["Tipo de Operacion"].nunique()

# Sólo hay un tipo de operación, lo cual es congruente, todas las filas son ventas.

1

In [132]:
# Finalmente convertimos la columna a string

df["Tipo de Operacion"] = df["Tipo de Operacion"].astype(str)


** **Verificar tipo de Inmuebles** **

In [133]:
# Verificar los tipos de inmuebles.

df["Tipo de Inmueble"].unique()

# Similarmente, hay dos opciones en el dataset: Apartamento y Casa. Esto es congruente.

array(['Apartamento', 'Casa'], dtype=object)

In [134]:
# Finalmente convertimos la columna a string

df["Tipo de Inmueble"] = df["Tipo de Inmueble"].astype(str)


** **Verificar Departamento** **

In [135]:
df["Departamento"].nunique()

# Aquí vemos algo extraño... Hay 4 departamentos, cuando debería haber uno. Veamos cuáles son...

4

In [136]:
df["Departamento"].unique()

# Hay datos del Magdalena, de Bogotá y de Cundinamarca. Veamos cuántos son.

array(['Antioquia', 'Magdalena', 'Bogotá, d.c.', 'Cundinamarca'],
      dtype=object)

In [137]:
df[df["Departamento"] != "Antioquia"]

# Interesante... Afortunadamente sólo son 4 de las filas, estos apartamentos existen 

,ID de la propiedad,Tipo de Operacion,Tipo de Inmueble,Departamento,Ciudad,Localidad Principal,Tipo de Localidad Principal,Metros Cuadrados,Metros Cuadrados Construidos,Estrato,...,Es Oficina?,Es Penthouse?,Acepta Permuta?,(paid quota)?,Fecha de Creacion de Oferta,Fecha de Actualización de Oferta,Dirección,URL,Precio con Administración Incluida,Precio
559,192532640,Venta,Casa,Magdalena,Bello,Bello,city,116.0,116.0,3,...,False,False,False,True,2025-06-15,2025-10-22,Casa en Venta en Bello,/casa-en-venta-en-bello/192532640,255147000,255000000
1197,192712949,Venta,Apartamento,"Bogotá, d.c.",Sabaneta,Sabaneta,city,57.0,57.0,4,...,False,False,False,True,2025-08-02,2025-08-04,Apartamento en Venta en Sabaneta,/apartamento-en-venta-en-sabaneta/192712949,377330000,377000000
1218,192696286,Venta,Apartamento,"Bogotá, d.c.",Sabaneta,Sabaneta,city,76.0,76.0,4,...,False,False,False,True,2025-07-29,2025-09-05,Apartamento en Venta en Sabaneta,/apartamento-en-venta-en-sabaneta/192696286,610450000,610000000
2071,193032462,Venta,Casa,Cundinamarca,Medellín,Medellín,city,186.0,186.0,4,...,False,False,False,True,2025-10-27,2025-10-28,Casa en Venta en Medellín,/casa-en-venta-en-medellin/193032462,910457000,910000000


In [138]:
# Borraré estos datos pues, luego de verificarlos manualmente, no corresponden al área metropolitana
# NOTA: Para verificación manual puede googlear el URL respectivo asociado a esta columna.

df = df[df["Departamento"] == "Antioquia"]

In [139]:
df["Departamento"].unique()

# Ahora sí, todo en orden...

array(['Antioquia'], dtype=object)

In [140]:
# Finalmente convertimos la columna a string

df["Departamento"] = df["Departamento"].astype(str)


** **Verificar Ciudad** **

In [141]:
df["Ciudad"].unique()

# Definitivamente hay ciudades que no corresponden a las ciudades especificadas anteriormente como campo de estudio.

# Como están fuera de la población estudiada, se eliminarán, pero antes veamos cuántas muestras erroneas tenemos

array(['Bello', 'Medellín', 'Copacabana', 'Acevedo', 'Santa ana',
       'La mesa', 'La ceja', 'La primavera', 'Sabaneta', 'San jose',
       'La estrella', 'Envigado', 'Jardín', 'Villanueva', 'Miraflores',
       'Itaguí', 'Santa maria', 'Rionegro', 'La paz', 'El Retiro',
       'Guatapé', 'Caldas', 'Manizales', 'San martin'], dtype=object)

In [142]:
ciudades = ("Medellín", "Envigado", "Sabaneta", "Itaguí", "La estrella", "Bello")

In [143]:
df[~ df["Ciudad"].isin(ciudades)]["Ciudad"].count()

# Como se observa, no son muchas.

# Pero antes, verifiquemos que los valores de ciudad unicos de este grupo no contengan estas ciudades
# (Tal vez un error de escritura de valores de nuestra parte)

np.int64(55)

In [144]:
df[~ df["Ciudad"].isin(ciudades)]["Ciudad"].unique()

# Ahora es claro que las ciudades que quedan no son las estudiadas. Procedemos a eliminarlas de los datos.

array(['Copacabana', 'Acevedo', 'Santa ana', 'La mesa', 'La ceja',
       'La primavera', 'San jose', 'Jardín', 'Villanueva', 'Miraflores',
       'Santa maria', 'Rionegro', 'La paz', 'El Retiro', 'Guatapé',
       'Caldas', 'Manizales', 'San martin'], dtype=object)

In [145]:
df[df["Ciudad"].isin(ciudades)]["Ciudad"].unique()

array(['Bello', 'Medellín', 'Sabaneta', 'La estrella', 'Envigado',
       'Itaguí'], dtype=object)

In [146]:
df = df[df["Ciudad"].isin(ciudades)]

** **Análisis valores únicos Tipo de Localidad Principal** **


In [147]:
# Comenzaré analizando los valores del nombre de localidad principal

df["Tipo de Localidad Principal"].unique()

# Si es neighbourhood, localidad principal es el nombre del barrio donde la propiedad se ubica.
# Si es city, es la ciudad donde está ubicada (analizaremos si esto es cierto contrastando con city)
# Si es zona, es la zone.
# None es no especificado.

array(['neighbourhood', 'city', 'commune', 'zone', None], dtype=object)

In [148]:
df[df["Tipo de Localidad Principal"] == "city"][["Localidad Principal", "Tipo de Localidad Principal"]]

# Si es ciudad, es la ciudad donde la propiedad está ubicada

,Localidad Principal,Tipo de Localidad Principal
2,Bello,city
3,Bello,city
7,Medellín,city
8,Bello,city
9,Bello,city
...,...,...
5772,La estrella,city
5778,La estrella,city
5803,La estrella,city
5804,La estrella,city


In [149]:
#  Análogamente....

df[df["Tipo de Localidad Principal"] == "commune"][["Localidad Principal", "Tipo de Localidad Principal"]]

,Localidad Principal,Tipo de Localidad Principal
15,Comuna 10,commune
55,Comuna 9,commune
82,Comuna 10,commune
112,Comuna 3,commune
138,Comuna 3,commune
...,...,...
3881,Comuna 3,commune
4027,Comuna 4,commune
4432,Comuna 4,commune
4481,Comuna 4,commune


In [150]:
df[df["Tipo de Localidad Principal"] == "zone"][["Localidad Principal", "Tipo de Localidad Principal"]]

,Localidad Principal,Tipo de Localidad Principal
1389,Suroriente,zone
2219,Centro,zone
2278,Centro,zone
2311,Suroriente,zone
2438,Centro,zone
2529,Centro,zone
3465,Suroriente,zone
3476,Suroriente,zone
4264,Suroriente,zone
4279,Suroriente,zone


In [151]:
df[df["Tipo de Localidad Principal"] == "neighbourhood"][["Localidad Principal", "Tipo de Localidad Principal"]]

#

,Localidad Principal,Tipo de Localidad Principal
0,Niquia,neighbourhood
1,Andalucia,neighbourhood
4,Prado,neighbourhood
5,Zona industrial 1,neighbourhood
6,Cabañas,neighbourhood
...,...,...
5822,La ferreira,neighbourhood
5823,Suramerica,neighbourhood
5824,La Tablaza,neighbourhood
5825,La ferreria,neighbourhood


In [152]:
df[df["Tipo de Localidad Principal"] == "neighbourhood"][["Localidad Principal", "Tipo de Localidad Principal"]].iloc[:, 0].unique()

# Visualmente se analiza que son barrios y no, como podría pasar, el nombre de la ciudad mal catalogado. 

array(['Niquia', 'Andalucia', 'Prado', 'Zona industrial 1', 'Cabañas',
       'Centro', 'San gabriel', 'Las vegas', 'La Navarra', 'Madera',
       'Santa ana', 'Sector belvedere', 'Nazareth', 'Buenos aires',
       'Zona industrial 3', 'Cabañitas', 'Fabricato', 'El trapiche',
       'El rosario', 'Parque tulio ospina', 'Bellavista', 'La gabriela',
       'Bucaros', 'Villas del sol', 'Paris', 'Altavista', 'Salento',
       'Hato viejo', 'El mirador', 'Barrio Nuevo',
       'Urbanizacion amazonia', 'El paraiso', 'Fontidueño', 'Los alpes',
       'San jose obrero', 'Urbanizacion palmar de serramonte',
       'Hospital mental', 'Playa rica', 'Laureles', 'El carmelo',
       'La primavera', 'Porvenir', 'San Antonio de Pereira', 'Quitasol',
       'Pachelly', 'Altos de quitasol', 'Asentamiento la orquidea',
       'Belén Los Alpes', 'Suarez', 'Los sauces', 'Carmelo', 'San martin',
       'San Felix', 'Molinares', 'Manchester', 'El Poblado', 'Terranova',
       'Alcala', 'Proyecto terranova',

In [153]:
# ¿ Qué se hará con esta columna? Idealmente diría nos daría la información más espceífica posible, en este caso barrio, 
# pero como se ha visto hay bastantes casos en los que esto no pasa. Contemos en cuántos no...

df[~(df["Tipo de Localidad Principal"] == "neighbourhood")][["Localidad Principal", "Tipo de Localidad Principal"]].count()

# En 1144 instancias no se provee esta información (y eso sin hacer análisis de consistencia)

# Se prevee buscar otra forma de hallar los barrios, hasta ahora se ha analizado la posibilidad de usar Nomatim, una API de geolocalización
# reversa gratuita. Consúltese: https://operations.osmfoundation.org/policies/nominatim/

# Si esto no funciona, se prevee extrapolar la información de barrio a partir de las coordenadas de longitud y latitud basándonos en los
# datos ya recopilados.

Localidad Principal            1144
Tipo de Localidad Principal    1144
dtype: int64

** **Análisis de la variable "Es Proyecto?"** **

In [154]:
df["Es Proyecto?"].unique()

# Es proyecto toma valores nulos o 1. 

array([nan,  1.])

In [155]:
df["Es Proyecto?"].sum()
# Como se evidencia, sólo en 12 casos toma valores 1. Es decir, el resto son Nulos.

np.float64(12.0)

In [159]:
# Al haber tantos nulos, se decide eliminar la columna

df.drop(columns=["Es Proyecto?"], inplace=True)

** **Análisis de fechas?"** **

El formato para cada variable parece ser el correcto, sin embargo hay que revisar variable por variable su formato. Las fechas parecen estar bien.
A continuación se muestran las columnas que tienen formato de fecha, y que formato tienen.

In [162]:
date_formats_report = []

for col in df.columns:
    if df[col].notna().sum() == 0:
        continue
    vals = df[col].dropna().astype(str)
    samples = vals.sample(min(50, len(vals)), random_state=1)
    parsed = pd.to_datetime(samples, errors='coerce', infer_datetime_format=True)
    if parsed.notna().mean() > 0.6:  # >60% parecen fechas
        formats = set()
        for v in samples:
            try:
                if "-" in v:
                    parts = v.split("-")
                    if len(parts)==3 and len(parts[0])==4:
                        formats.add("YYYY-MM-DD")
                    elif len(parts)==3 and len(parts[2])==4:
                        formats.add("DD-MM-YYYY")
                    else:
                        formats.add("other-with-dash")
                else:
                    formats.add("non-dash")
            except:
                formats.add("unknown")
        date_formats_report.append((col, list(formats)))

date_formats_report

[('Fecha de Creacion de Oferta', ['YYYY-MM-DD']),
 ('Fecha de Actualización de Oferta', ['YYYY-MM-DD'])]

In [45]:
df[col].astype(str).str.contains("[^0-9\\.-]").any()

np.False_

No hay cosas raras en las columnas de string.

#### Consistencia

In [46]:
df[(df['Dormitorios (bedrooms)']> df['Cuartos (rooms)']  )]

,ID de la propiedad,Tipo de Operacion,Tipo de Inmueble,Departamento,Ciudad,Localidad Principal,Tipo de Localidad Principal,Metros Cuadrados,Metros Cuadrados Construidos,Estrato,...,Es Oficina?,Es Penthouse?,Acepta Permuta?,(paid quota)?,Fecha de Creacion de Oferta,Fecha de Actualización de Oferta,Dirección,URL,Precio con Administración Incluida,Precio
0,192788587,Venta,Apartamento,Antioquia,Bello,Niquia,neighbourhood,59.00,NaN,3,...,False,False,False,True,2025-08-25,2025-09-16,"Apartamento en Venta en Niquia, Bello",/apartamento-en-venta-en-niquia-bello/192788587,275000000,275000000
5,192868806,Venta,Apartamento,Antioquia,Bello,Zona industrial 1,neighbourhood,47.31,47.31,4,...,False,False,False,True,2025-09-13,2025-09-13,"Apartamento en Venta en Zona industrial 1, Bello",/apartamento-en-venta-en-zona-industrial-1-bel...,320000000,320000000
14,192886436,Venta,Apartamento,Antioquia,Bello,Cabañas,neighbourhood,80.00,80.00,4,...,False,False,False,False,2025-09-17,2025-09-17,"Apartamento en Venta en Cabañas, Bello",/apartamento-en-venta-en-cabañas-bello/192886436,275000000,275000000
24,191997110,Venta,Apartamento,Antioquia,Bello,Sector belvedere,neighbourhood,55.60,55.60,3,...,False,False,False,True,2025-02-10,2025-07-04,"Apartamento en Venta en Sector belvedere, Bello",/apartamento-en-venta-en-sector-belvedere-bell...,300220000,300000000
25,192961362,Venta,Casa,Antioquia,Bello,Cabañas,neighbourhood,98.00,NaN,4,...,False,False,False,True,2025-10-08,2025-10-15,"Casa en Venta en Cabañas, Bello",/casa-en-venta-en-cabañas-bello/192961362,425000000,425000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5812,193001267,Venta,Apartamento,Antioquia,La estrella,La Tablaza,neighbourhood,53.00,NaN,3,...,False,False,False,True,2025-10-20,2025-10-31,"Apartamento en Venta en La Tablaza, La estrella",/apartamento-en-venta-en-la-tablaza-la-estrell...,237210000,237000000
5813,193002785,Venta,Apartamento,Antioquia,La estrella,Pueblo Viejo,neighbourhood,51.00,NaN,3,...,False,False,False,True,2025-10-20,2025-10-30,"Apartamento en Venta en Pueblo Viejo, La estrella",/apartamento-en-venta-en-pueblo-viejo-la-estre...,230152000,230000000
5815,193001396,Venta,Apartamento,Antioquia,La estrella,La ferreria,neighbourhood,69.00,69.00,4,...,False,False,False,True,2025-10-20,2025-10-20,"Apartamento en Venta en La ferreria, La estrella",/apartamento-en-venta-en-la-ferreria-la-estrel...,377300000,377000000
5816,193003287,Venta,Apartamento,Antioquia,Caldas,La Tablaza,neighbourhood,38.00,NaN,2,...,False,False,False,True,2025-10-20,2025-10-23,"Apartamento en Venta en La Tablaza, Caldas",/apartamento-en-venta-en-la-tablaza-caldas/193...,200140000,200000000


Registros donde hay mas dormitorios que cuartos, lo cual se contradice

In [47]:
df[df['Metros Cuadrados'] < df['Metros Cuadrados Construidos'] ]

,ID de la propiedad,Tipo de Operacion,Tipo de Inmueble,Departamento,Ciudad,Localidad Principal,Tipo de Localidad Principal,Metros Cuadrados,Metros Cuadrados Construidos,Estrato,...,Es Oficina?,Es Penthouse?,Acepta Permuta?,(paid quota)?,Fecha de Creacion de Oferta,Fecha de Actualización de Oferta,Dirección,URL,Precio con Administración Incluida,Precio


In [61]:
from scipy.stats import chi2

# seleccionar variables numéricas relevantes
X = df[["Precio", "Metros Cuadrados", "Cuartos (rooms)", "Baños"]].dropna()

# calcular media y covarianza
mu = X.mean(axis=0)
cov = np.cov(X.values, rowvar=False)
inv_cov = np.linalg.inv(cov)

# distancia de Mahalanobis
diff = X - mu
md = np.sqrt(np.diag(diff @ inv_cov * diff.T))

# umbral según chi-cuadrado
threshold = np.sqrt(chi2.ppf(0.99, df=X.shape[1]))

# puntos incoherentes
inconsistentes = X[md > threshold]
print(X)
print(inconsistentes)

         Precio  Metros Cuadrados  Cuartos (rooms)  Baños
0     275000000              59.0                0      2
1     280000000              64.9                0      2
2     260000000              67.0                3      2
3     433000000              72.0                3      2
4     295000000              91.0                3      2
...         ...               ...              ...    ...
5822  350000000              53.0                2      2
5823  890000000             147.0                3      3
5824  230000000              48.0                2      1
5825  528000000              78.0                3      2
5826  350000000              53.0                0      2

[5785 rows x 4 columns]
Empty DataFrame
Columns: [Precio, Metros Cuadrados, Cuartos (rooms), Baños]
Index: []


No parece haber datos salidos de la elipse de mahalanobis

#### Precisión/Exactitud:

In [32]:
df[( df['Estrato']< 1 ) | (df['Estrato'] > 6 )] 

,ID de la propiedad,Tipo de Operacion,Tipo de Inmueble,Departamento,Ciudad,Localidad Principal,Tipo de Localidad Principal,Metros Cuadrados,Metros Cuadrados Construidos,Estrato,...,Es Oficina?,Es Penthouse?,Acepta Permuta?,(paid quota)?,Fecha de Creacion de Oferta,Fecha de Actualización de Oferta,Dirección,URL,Precio con Administración Incluida,Precio
15,192945237,Venta,Apartamento,Antioquia,Bello,Comuna 10,commune,61.0,61.0,0,...,False,False,False,True,2025-10-03,2025-10-22,"Apartamento en Venta en Comuna 10, Bello",/apartamento-en-venta-en-comuna-10-bello/19294...,295224840,295000000
31,192010579,Venta,Apartamento,Antioquia,Bello,Centro,neighbourhood,58.0,58.0,0,...,False,False,False,True,2025-02-12,2025-10-22,"Apartamento en Venta en Centro, Bello",/apartamento-en-venta-en-centro-bello/192010579,305230000,305000000
34,192020844,Venta,Apartamento,Antioquia,Bello,Buenos aires,neighbourhood,55.0,55.0,0,...,False,False,False,True,2025-02-13,2025-10-22,"Apartamento en Venta en Buenos aires, Bello",/apartamento-en-venta-en-buenos-aires-bello/19...,250255400,250000000
37,192010581,Venta,Apartamento,Antioquia,Bello,Cabañitas,neighbourhood,93.0,93.0,0,...,False,False,False,True,2025-02-12,2025-10-22,"Apartamento en Venta en Cabañitas, Bello",/apartamento-en-venta-en-cabañitas-bello/19201...,420100000,420000000
63,192971088,Venta,Apartamento,Antioquia,Bello,Cabañas,neighbourhood,55.0,55.0,110,...,False,False,False,True,2025-10-10,2025-10-10,"Apartamento en Venta en Cabañas, Bello",/apartamento-en-venta-en-cabañas-bello/192971088,275200000,275000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5654,192846264,Venta,Casa,Antioquia,La estrella,Poblado Del Sur,neighbourhood,137.0,137.0,0,...,False,False,False,True,2025-09-08,2025-10-22,"Casa en Venta en Poblado Del Sur, La estrella",/casa-en-venta-en-poblado-del-sur-la-estrella/...,750353000,750000000
5741,192921734,Venta,Casa,Antioquia,La estrella,Pueblo Viejo,neighbourhood,865.0,865.0,0,...,False,False,False,True,2025-09-26,2025-10-22,"Casa en Venta en Pueblo Viejo, La estrella",/casa-en-venta-en-pueblo-viejo-la-estrella/192...,959000000,959000000
5762,192985963,Venta,Apartamento,Antioquia,La estrella,Pueblo Viejo,neighbourhood,76.0,NaN,0,...,False,False,False,True,2025-10-15,2025-10-23,"Apartamento en Venta en Pueblo Viejo, La estrella",/apartamento-en-venta-en-pueblo-viejo-la-estre...,530270000,530000000
5779,192968035,Venta,Apartamento,Antioquia,La estrella,Yarumito,neighbourhood,87.0,87.0,0,...,False,False,False,True,2025-10-09,2025-10-22,"Apartamento en Venta en Yarumito, La estrella",/apartamento-en-venta-en-yarumito-la-estrella/...,670350000,670000000


Encontramos que el hay 236 registros donde sus estratos no corresponden a los estandarizados por el gobierno, es decir registros erróneos

In [65]:
df[(df['Precio'] <= 100000000) |  (df['Precio'] > 100000000000)]

,ID de la propiedad,Tipo de Operacion,Tipo de Inmueble,Departamento,Ciudad,Localidad Principal,Tipo de Localidad Principal,Metros Cuadrados,Metros Cuadrados Construidos,Estrato,...,Es Oficina?,Es Penthouse?,Acepta Permuta?,(paid quota)?,Fecha de Creacion de Oferta,Fecha de Actualización de Oferta,Dirección,URL,Precio con Administración Incluida,Precio
2044,192989048,Venta,Apartamento,Antioquia,Medellín,El chagualo,neighbourhood,40.00,NaN,3,...,False,False,False,True,2025-10-16,2025-10-24,"Apartamento en Venta en El chagualo, Medellín",/apartamento-en-venta-en-el-chagualo-medellin/...,250000000000,250000000000
2477,193001539,Venta,Apartamento,Antioquia,Medellín,El Poblado,neighbourhood,76.00,76.00,5,...,False,False,False,True,2025-10-20,2025-10-20,"Apartamento en Venta en El Poblado, Medellín",/apartamento-en-venta-en-el-poblado-medellin/1...,474000000000,474000000000
3365,192451446,Venta,Apartamento,Antioquia,Itaguí,Itaguí,city,54.00,54.00,4,...,False,False,False,True,2025-05-26,2025-05-26,Apartamento en Venta en Itaguí,/apartamento-en-venta-en-itagui/192451446,347000192200,347000000000
5244,192228606,Venta,Casa,Antioquia,La estrella,La estrella,city,159.00,159.00,4,...,False,False,False,True,2025-04-03,2025-08-29,Casa en Venta en La estrella,/casa-en-venta-en-la-estrella/192228606,390001,1
5362,192570795,Venta,Apartamento,Antioquia,La estrella,Suramérica,neighbourhood,87.00,87.00,4,...,False,False,False,True,2025-06-26,2025-09-04,"Apartamento en Venta en Suramérica, La estrella",/apartamento-en-venta-en-suramerica-la-estrell...,630000340000,630000000000
5474,192715365,Venta,Apartamento,Antioquia,La estrella,La estrella,city,50.27,50.27,3,...,False,False,False,False,2025-08-04,2025-08-25,Apartamento en Venta en La estrella,/apartamento-en-venta-en-la-estrella/192715365,55000000,55000000


Curioso, dos casas muy baratas! Una de 1 peso y una de 55 millones. Ademas 4 casas sorprendentemente caras, seguramente sean precios erróneos

In [70]:
df['Cuartos (rooms)'].unique()

array([ 0,  3,  2,  4,  1,  6,  5,  7,  8, 10, 12, 15])

In [71]:
df['Baños'].unique()

array([ 2,  1,  3,  5,  4,  0,  6,  7,  9,  8, 10])

In [72]:
df['Dormitorios (bedrooms)'].unique()

array([ 3,  0,  1,  2,  4,  6,  5,  7,  8, 10, 12, 15])

Seguramente hay casas de 15 baños, pero quizas se pueda revisar. Se ve por el analisis de mahalanobis que estos numeros combinan con los metros cuadrados.

#### Duplicados 

In [75]:
df.duplicated().sum()

np.int64(170)

Hay 170 registros duplicados, sin embargo quiero ver ademas segun el ID cuantos registros hay duplicados, y con el url

In [77]:
df['URL'].duplicated().sum()

np.int64(170)

In [78]:
df['ID de la propiedad'].duplicated().sum()

np.int64(170)

#### Integridad

Debido a la fuente de los datos, no es necesario hacer un análisis de integridad

In [7]:
df.describe()

,ID de la propiedad,Metros Cuadrados,Metros Cuadrados Construidos,Estrato,Latitud,Longitud,Antigüedad (Grupo),Cuartos (rooms),Dormitorios (bedrooms),Baños,Parqueaderos,Piso,Es Proyecto?,Precio con Administración Incluida,Precio
count,5.827000e+03,5785.000000,4828.000000,5827.000000,5827.000000,5827.000000,5703.000000,5827.000000,5827.000000,5827.000000,5827.000000,5612.000000,13.0,5.827000e+03,5.827000e+03
mean,1.894227e+08,149.676068,130.790387,4.167496,6.199873,-75.596745,2.285113,2.258109,2.889995,2.382015,1.143985,5.257662,1.0,1.007876e+09,1.005765e+09
std,2.453440e+07,1777.493584,324.867469,6.239423,0.100346,0.050384,1.314624,1.451103,0.871457,1.001855,1.183491,6.845410,0.0,1.175982e+10,1.175971e+10
min,5.093501e+06,0.000000,0.000000,0.000000,4.620578,-75.661438,0.000000,0.000000,0.000000,0.000000,-2.000000,-1.000000,1.0,3.900010e+05,1.000000e+00
25%,1.926759e+08,62.000000,62.000000,3.000000,6.153324,-75.625082,2.000000,2.000000,2.000000,2.000000,0.000000,0.000000,1.0,3.500000e+08,3.500000e+08
50%,1.928929e+08,76.000000,78.000000,4.000000,6.167528,-75.601615,2.000000,3.000000,3.000000,2.000000,1.000000,2.000000,1.0,4.922600e+08,4.920000e+08
75%,1.929936e+08,114.000000,118.000000,4.000000,6.225058,-75.569930,3.000000,3.000000,3.000000,2.000000,1.000000,10.000000,1.0,7.600000e+08,7.535000e+08
max,1.930546e+08,133377.000000,11000.000000,110.000000,11.212865,-74.030012,5.000000,15.000000,15.000000,10.000000,14.000000,97.000000,1.0,6.300003e+11,6.300000e+11


Se ven cosas curiosas, como que el estrato de uno de los registros sea 110. 

De resto, las otras columnas aparentan ser normales, aunque los metros cuadrados y los pisos tienen una excepción extraña, donde el máximo parece ser de 133377. Eso es un resultado un poco loco, dado en cuenta que el tercer cuartil es de 114, y el piso máximo es el 97 (no se que edificio tiene 97 pisos en el valle de aburrá) y el mínimo es el -1, que podría ser un sótano o un error. Un precio máximo de 6.3 e11 también es muy muy muy elevado, altamente probable que este sea una anomalía.
